### Importing Data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('/content/LinkedIn people profiles datasets.csv')


### Data Exploring

In [33]:
df.head()

,name,city,country_code,position,current_company,educations_details,field,title,degree,meta,start_year,end_year,company,position_titles,position_durations
0,"Catherine Fitzpatrick (McIlkenny), B.A",Canada,CA,Snr Business Analyst at Emploi et Développemen...,,Queen's University Belfast,English Language and Literature,Queen's University Belfast,Bachelor of Arts (B.A.) Honours,2009 - 2011,2009,2011,None,None,None
1,Margot Bon,"The Randstad, Netherlands",NL,Communicatieadviseur Corporate & Strategie Gem...,Gemeente Utrecht,NaN,,Xebia Academy International,Scrum en Agile werken,2019 - 2019,2019,2019,Gemeente Utrecht,[Communicatieadviseur Corporate & Strategie],[May 2022 - Present 8 months]
2,Mike Dean,"England, United Kingdom",UK,Network Data Manager at Network Rail,Network Rail,Brighton Polytechnic,Electronic Engineering,Brighton Polytechnic,2:2,1988 - 1991,1988,1991,Network Rail,"[Network Data Manager LNE & EM, Infrastructure...","[Apr 2019 - Present 4 years 2 months, Jan 2013..."
3,Giovanna Panarella,"Avellino, Campania, Italy",IT,Architetto (Freelance),Freelance,Università di Camerino,Architettura,Università di Camerino,“Corso di aggiornamento professionale in BIM S...,2020 - 2021,2020,2021,Freelance,[Architetto],[May 2020 - Present 2 years 1 month]
4,Steve Latimer,"Ontario, Canada",CA,Senior Account Executive at Mid-Range Computer...,Mid-Range Computer Group Inc.,St. Michael's College School,,St. Michael's College School,,1973 - 1978,1973,1978,Mid-Range Computer Group Inc.,[Senior Account Executive],[1996 - Present 26 years]


In [34]:
df['id'].value_counts, df.shape

KeyError: ignored

Length -> 1000
df.shape -> (1000, 26), we do not have any duplicated rows,we are good to go. We would drop them if we had.

In [ ]:
df.columns

In [ ]:
if df['country_code'].isna().any():
  df.dropna(subset=['country_code'], inplace=True)
  print(f"Number of empty values after dropping = {df['country_code'].isna().sum()}")

In [ ]:
df['country_code'].unique(), len(df["country_code"].unique())

we have profiles from 82 different countries

### Data Cleaning

In [8]:
df.drop(["volunteer_experience", "current_company:name", "current_company:company_id", "сourses", "recommendations_count", "recommendations", "languages", "avatar", "people_also_viewed", "url", "groups", "posts", "about", "region", "id", "timestamp"], axis=1, inplace=True)

I drop these columns because they either have no useful information nor have NaN values

In [9]:
df["certifications"].isna().sum(), df.shape

(815, (998, 10))

In [10]:
df["following"].isna().sum(), df.shape

(517, (998, 10))

we have 1000 rows but 816 of them have no "certifications", 519 of them have no "following", so I will drop the columns

In [11]:
df.drop(['certifications', "following"], axis=1, inplace=True)

In [12]:
df.head()

,name,city,country_code,position,current_company,experience,educations_details,education
0,"Catherine Fitzpatrick (McIlkenny), B.A",Canada,CA,Snr Business Analyst at Emploi et Développemen...,"{""name"":""""}",NaN,Queen's University Belfast,"[{""degree"":""Bachelor of Arts (B.A.) Honours"",""..."
1,Margot Bon,"The Randstad, Netherlands",NL,Communicatieadviseur Corporate & Strategie Gem...,"{""company_id"":""gemeente-utrecht"",""industry"":""G...","[{""company"":""Gemeente Utrecht"",""company_id"":""g...",NaN,"[{""degree"":""Scrum en Agile werken"",""end_year"":..."
2,Mike Dean,"England, United Kingdom",UK,Network Data Manager at Network Rail,"{""company_id"":""network-rail"",""link"":""https://s...","[{""company"":""Network Rail"",""company_id"":""netwo...",Brighton Polytechnic,"[{""degree"":""2:2"",""end_year"":""1991"",""field"":""El..."
3,Giovanna Panarella,"Avellino, Campania, Italy",IT,Architetto (Freelance),"{""link"":null,""name"":""Freelance""}","[{""company"":""Freelance"",""company_id"":null,""loc...",Università di Camerino,"[{""degree"":""“Corso di aggiornamento profession..."
4,Steve Latimer,"Ontario, Canada",CA,Senior Account Executive at Mid-Range Computer...,"{""company_id"":""mid-range-computer-group-inc."",...","[{""company"":""Mid-Range Computer Group Inc."",""c...",St. Michael's College School,"[{""degree"":"""",""end_year"":""1978"",""field"":"""",""me..."


### Extract Company Names

In [13]:
df['current_company'].iloc[:]

0                                            {"name":""}
1      {"company_id":"gemeente-utrecht","industry":"G...
2      {"company_id":"network-rail","link":"https://s...
3                       {"link":null,"name":"Freelance"}
4      {"company_id":"mid-range-computer-group-inc.",...
                             ...                        
995             {"link":null,"name":"Awwads Lernzirkel"}
996    {"company_id":"ball-aerospace","industry":"Def...
997                                          {"name":""}
998                                          {"name":""}
999                {"link":null,"name":"Collier County"}
Name: current_company, Length: 998, dtype: object

In [14]:
import json

def extract_name(row):
    try:
        json_data = json.loads(row)
        return json_data.get('name', '') # If 'name' is found in the dictionary, it returns its value. if name not found, return empty
    except json.JSONDecodeError:
        return ''

In [15]:
df['current_company'] = df['current_company'].apply(extract_name)

### Extract Education Information

In [16]:
df['education'].iloc[2]

'[{"degree":"2:2","end_year":"1991","field":"Electronic Engineering","meta":"1988 - 1991","start_year":"1988","title":"Brighton Polytechnic"}]'

In [17]:
def extract_education_info(education_json):
    if pd.isna(education_json):
        return None, None, None, None, None, None
    education_data = json.loads(education_json)
    if len(education_data) > 0:
        education_info = education_data[0]  # Assuming there is only one education entry per row
        return education_info.get('field'), education_info.get('title'), education_info.get('degree'), education_info.get('meta'), education_info.get('start_year'), education_info.get('end_year')
    else:
        return None, None, None, None, None, None

In [18]:
df.head(1)

,name,city,country_code,position,current_company,experience,educations_details,education
0,"Catherine Fitzpatrick (McIlkenny), B.A",Canada,CA,Snr Business Analyst at Emploi et Développemen...,,NaN,Queen's University Belfast,"[{""degree"":""Bachelor of Arts (B.A.) Honours"",""..."


In [19]:
# Apply the function to the 'education' column
df['field'], df['title'], df['degree'], df['meta'], df['start_year'], df['end_year'] = zip(*df['education'].apply(extract_education_info))

# Drop the 'education' column if no longer needed
df.drop('education', axis=1, inplace=True)

# Print the resulting DataFrame
df.head(1)

,name,city,country_code,position,current_company,experience,educations_details,field,title,degree,meta,start_year,end_year
0,"Catherine Fitzpatrick (McIlkenny), B.A",Canada,CA,Snr Business Analyst at Emploi et Développemen...,,NaN,Queen's University Belfast,English Language and Literature,Queen's University Belfast,Bachelor of Arts (B.A.) Honours,2009 - 2011,2009,2011


In [20]:
df.head(3)

,name,city,country_code,position,current_company,experience,educations_details,field,title,degree,meta,start_year,end_year
0,"Catherine Fitzpatrick (McIlkenny), B.A",Canada,CA,Snr Business Analyst at Emploi et Développemen...,,NaN,Queen's University Belfast,English Language and Literature,Queen's University Belfast,Bachelor of Arts (B.A.) Honours,2009 - 2011,2009,2011
1,Margot Bon,"The Randstad, Netherlands",NL,Communicatieadviseur Corporate & Strategie Gem...,Gemeente Utrecht,"[{""company"":""Gemeente Utrecht"",""company_id"":""g...",NaN,,Xebia Academy International,Scrum en Agile werken,2019 - 2019,2019,2019
2,Mike Dean,"England, United Kingdom",UK,Network Data Manager at Network Rail,Network Rail,"[{""company"":""Network Rail"",""company_id"":""netwo...",Brighton Polytechnic,Electronic Engineering,Brighton Polytechnic,2:2,1988 - 1991,1988,1991


In [21]:
df["experience"].iloc[1]

'[{"company":"Gemeente Utrecht","company_id":"gemeente-utrecht","industry":"Government Administration","location":"Utrecht, Netherlands","positions":[{"description":"Bijdragen aan de positionering en profilering van Gemeente Utrecht als stad van Gezond Stedelijk Leven voor Iedereen. Verder ontwikkelen Corporate Communicatie (afdeling) en Corporate Story. Opzetten en uitvoeren van (interne) communicatiestrategie, plan en middelen. Versterken communicatiekracht van de organisatie. Communicatieadviseur voor interne- en corporate projecten o.a. Arbeidsmarkt Campagne, nieuw intranet, talkshow Studio030, DUS, De Utrechtse Standaard Bijdragen aan de positionering en profilering van Gemeente Utrecht als stad van Gezond Stedelijk Leven voor Iedereen. Verder ontwikkelen Corporate Communicatie (afdeling) en Corporate Story. Opzetten en uitvoeren van (interne) communicatiestrategie, plan en middelen. Versterken communicatiekracht van de organisatie. Communicatieadviseur voor interne- en corporate 

In [22]:
# company, duration, positions
def extract_position_info(positions_json):
    if pd.isna(positions_json):
        return None, None, None

    position_data = json.loads(positions_json)
    if len(position_data) > 0:
        positions_info = position_data[0]  # Assuming there is only one set of positions per row
        company = positions_info.get('company')
        duration = positions_info.get('duration')
        positions = positions_info.get('positions', [])

        position_titles = []
        position_durations = []
        position_metas = []

        for position in positions:
            position_titles.append(position.get('title'))
            position_durations.append(position.get('duration'))
            position_metas.append(position.get('meta'))

        return company,  position_titles, position_durations
    else:
        return None, None, None, None, None

# Apply the function to the 'positions' column
df['company'],  df['position_titles'], df['position_durations'] = zip(*df['experience'].apply(extract_position_info))

# Drop the 'positions' column if no longer needed
df.drop('experience', axis=1, inplace=True)

# Print the resulting DataFrame
df.head()

,name,city,country_code,position,current_company,educations_details,field,title,degree,meta,start_year,end_year,company,position_titles,position_durations
0,"Catherine Fitzpatrick (McIlkenny), B.A",Canada,CA,Snr Business Analyst at Emploi et Développemen...,,Queen's University Belfast,English Language and Literature,Queen's University Belfast,Bachelor of Arts (B.A.) Honours,2009 - 2011,2009,2011,None,None,None
1,Margot Bon,"The Randstad, Netherlands",NL,Communicatieadviseur Corporate & Strategie Gem...,Gemeente Utrecht,NaN,,Xebia Academy International,Scrum en Agile werken,2019 - 2019,2019,2019,Gemeente Utrecht,[Communicatieadviseur Corporate & Strategie],[May 2022 - Present 8 months]
2,Mike Dean,"England, United Kingdom",UK,Network Data Manager at Network Rail,Network Rail,Brighton Polytechnic,Electronic Engineering,Brighton Polytechnic,2:2,1988 - 1991,1988,1991,Network Rail,"[Network Data Manager LNE & EM, Infrastructure...","[Apr 2019 - Present 4 years 2 months, Jan 2013..."
3,Giovanna Panarella,"Avellino, Campania, Italy",IT,Architetto (Freelance),Freelance,Università di Camerino,Architettura,Università di Camerino,“Corso di aggiornamento professionale in BIM S...,2020 - 2021,2020,2021,Freelance,[Architetto],[May 2020 - Present 2 years 1 month]
4,Steve Latimer,"Ontario, Canada",CA,Senior Account Executive at Mid-Range Computer...,Mid-Range Computer Group Inc.,St. Michael's College School,,St. Michael's College School,,1973 - 1978,1973,1978,Mid-Range Computer Group Inc.,[Senior Account Executive],[1996 - Present 26 years]


### Analyze

In [23]:
df.head(6)

,name,city,country_code,position,current_company,educations_details,field,title,degree,meta,start_year,end_year,company,position_titles,position_durations
0,"Catherine Fitzpatrick (McIlkenny), B.A",Canada,CA,Snr Business Analyst at Emploi et Développemen...,,Queen's University Belfast,English Language and Literature,Queen's University Belfast,Bachelor of Arts (B.A.) Honours,2009 - 2011,2009,2011,None,None,None
1,Margot Bon,"The Randstad, Netherlands",NL,Communicatieadviseur Corporate & Strategie Gem...,Gemeente Utrecht,NaN,,Xebia Academy International,Scrum en Agile werken,2019 - 2019,2019,2019,Gemeente Utrecht,[Communicatieadviseur Corporate & Strategie],[May 2022 - Present 8 months]
2,Mike Dean,"England, United Kingdom",UK,Network Data Manager at Network Rail,Network Rail,Brighton Polytechnic,Electronic Engineering,Brighton Polytechnic,2:2,1988 - 1991,1988,1991,Network Rail,"[Network Data Manager LNE & EM, Infrastructure...","[Apr 2019 - Present 4 years 2 months, Jan 2013..."
3,Giovanna Panarella,"Avellino, Campania, Italy",IT,Architetto (Freelance),Freelance,Università di Camerino,Architettura,Università di Camerino,“Corso di aggiornamento professionale in BIM S...,2020 - 2021,2020,2021,Freelance,[Architetto],[May 2020 - Present 2 years 1 month]
4,Steve Latimer,"Ontario, Canada",CA,Senior Account Executive at Mid-Range Computer...,Mid-Range Computer Group Inc.,St. Michael's College School,,St. Michael's College School,,1973 - 1978,1973,1978,Mid-Range Computer Group Inc.,[Senior Account Executive],[1996 - Present 26 years]
5,Manuela Dias,"Madrid, Community of Madrid, Spain",ES,Operational Excellence & Internal Control Mana...,Ericsson,NaN,None,None,None,None,None,None,Ericsson,[Operational Excellence & Internal Control Man...,"[May 2016 - Present 6 years 8 months, Aug 2003..."


In [24]:
# Country, company, education_detals, field, title, longest position duration (start-end date)

In [25]:
df.to_csv('LinkedInCleaned.csv', index=False)

### PLOT

In [26]:
!pip install plotly

In [32]:
import pandas as pd
import plotly.express as px

# Assuming you already have a DataFrame 'df' with a 'country_code' column

# Count the occurrences of each 'country_code'
count_df = df['country_code'].value_counts().reset_index()
count_df.columns = ['country_code', 'count']

# Calculate the percentages
count_df['percentage'] = (count_df['count'] / count_df['count'].sum()) * 100

# Group values with percentages less than 3% into 'Others'
threshold = 2
count_df.loc[count_df['percentage'] < threshold, 'country_code'] = 'Others'
count_df = count_df.groupby('country_code', as_index=False)['count'].sum()

# Create a pie chart
fig = px.pie(count_df, names='country_code', values='count')

# Customize the pie chart (optional)
fig.update_traces(textposition='inside', textinfo='percent+label')

# Add a title (optional)
fig.update_layout(title_text='Pie Chart of Country Code Counts')

# Show the plot
fig.show()
